In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
import re
import time
import random
import warnings
from collections import Counter
import numpy as np, pandas as pd, polars as pl

import torch
import vllm
from vllm import LLM, SamplingParams

import kaggle_evaluation.aimo_2_inference_server

warnings.simplefilter('ignore')
print('PyTorch version:', torch.__version__)
print('vLLM:', vllm.__version__)

INFO 03-11 15:26:57 __init__.py:183] Automatically detected platform cuda.
PyTorch version: 2.5.1+cu124
vLLM: 0.7.1


In [2]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
seed_everything(seed=0)

start_time = time.time()
cutoff_time = start_time + (4 * 60 + 45) * 60
cutoff_times = [int(x) for x in np.linspace(cutoff_time, start_time + 60 * 60, 50 + 1)]

In [3]:
if os.getenv('KAGGLE_KERNEL_RUN_TYPE') or os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    llm_model_pth = '/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1'
else:
    llm_model_pth = '/root/volume/KirillR/QwQ-32B-Preview-AWQ'

MAX_NUM_SEQS = 32 #16
MAX_MODEL_LEN = 8192 * 3 // 2 #16384 #32768 #

llm = LLM(
    llm_model_pth,
#    dtype="half",                 # The data type for the model weights and activations
    max_num_seqs=MAX_NUM_SEQS,    # Maximum number of sequences per iteration. Default is 256
    max_model_len=MAX_MODEL_LEN,  # Model context length
    trust_remote_code=True,       # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
    tensor_parallel_size=4,       # The number of GPUs to use for distributed execution with tensor parallelism
    gpu_memory_utilization=0.95,  # The ratio (between 0 and 1) of GPU memory to reserve for the model
    seed=2024,
)

tokenizer = llm.get_tokenizer()

INFO 03-11 15:27:33 config.py:526] This model supports multiple tasks: {'reward', 'score', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 03-11 15:27:37 awq_marlin.py:109] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 03-11 15:27:37 config.py:1383] Defaulting to use mp for distributed inference
WARNING 03-11 15:27:37 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
INFO 03-11 15:27:37 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1', speculative_config=None, tokenizer='/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=12288, download_dir=None, load_format=auto, tensor_parallel

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(VllmWorkerProcess pid=354) INFO 03-11 15:28:25 model_runner.py:1116] Loading model weights took 1.3375 GB
INFO 03-11 15:28:25 model_runner.py:1116] Loading model weights took 1.3375 GB
(VllmWorkerProcess pid=351) INFO 03-11 15:28:25 model_runner.py:1116] Loading model weights took 1.3375 GB
(VllmWorkerProcess pid=359) INFO 03-11 15:28:25 model_runner.py:1116] Loading model weights took 1.3375 GB
(VllmWorkerProcess pid=354) (VllmWorkerProcess pid=359) WARNING 03-11 15:28:44 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
WARNING 03-11 15:28:44 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
(VllmWorkerProcess pid=354) (VllmWorkerProcess pid=359) WARNING 03-11 15:28:44 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
(VllmWorkerProcess pid=351) WARNING 03-11 15:28:44 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
(VllmWorkerProcess pid=354) WARNING 03-11 15

Capturing CUDA graph shapes:  86%|████████▌ | 6/7 [00:05<00:00,  1.08it/s]

(VllmWorkerProcess pid=351) (VllmWorkerProcess pid=354) INFO 03-11 15:28:57 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB
INFO 03-11 15:28:57 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB
(VllmWorkerProcess pid=359) 

Capturing CUDA graph shapes: 100%|██████████| 7/7 [00:07<00:00,  1.10s/it]

INFO 03-11 15:28:57 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB


Capturing CUDA graph shapes: 100%|██████████| 7/7 [00:07<00:00,  1.02s/it]

INFO 03-11 15:28:57 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB
INFO 03-11 15:28:57 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 31.84 seconds


In [4]:
def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""

def batch_message_filter(list_of_messages) -> tuple[list[list[dict]], list[str]]:
    extracted_answers = []
    list_of_messages_to_keep = []
    for messages in list_of_messages:
        answer = extract_boxed_text(messages[-1]['content'])
        if answer:
            extracted_answers.append(answer)
        else:
            list_of_messages_to_keep.append(messages)
    return list_of_messages_to_keep, extracted_answers

def select_answer(answers):
    counter = Counter()
    for answer in answers:
        try:
            if int(answer) == float(answer):
                counter[int(answer)] += 1 + random.random() / 1_000
        except:
            pass
    if not counter:
        return 210
    _, answer = sorted([(v,k) for k,v in counter.items()], reverse=True)[0]
    return answer%1000

def batch_message_generate(list_of_messages) -> list[list[dict]]:
    max_tokens = MAX_MODEL_LEN
    if time.time() > cutoff_times[-1]:
        print("Speedrun")
        max_tokens = 2 * MAX_MODEL_LEN // 3

    sampling_params = SamplingParams(
        temperature=0.6,               # Randomness of the sampling
        # top_p=0.90,                    # Cumulative probability of the top tokens to consider
        # min_p=0.05,                    # Minimum probability for a token to be considered
        skip_special_tokens=True,      # Whether to skip special tokens in the output
        max_tokens=max_tokens,         # Maximum number of tokens to generate
        stop=["</think>"],             # List of strings that stop the generation
        seed=2025,
    )
    
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]

    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    print([len(single_request_output.outputs[0].token_ids) for single_request_output in request_output])

    sort_keys_and_list_of_messages = []
    for messages, single_request_output in zip(list_of_messages, request_output):
        #print()
        #print(single_request_output.outputs[0].text)
        #print()
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})

        sort_keys_and_list_of_messages.append(
            (
                len(single_request_output.outputs[0].token_ids),
                messages
            )
        )
    print([sort_key for sort_key, _ in sort_keys_and_list_of_messages])
    sort_keys_and_list_of_messages.sort(key=lambda sort_key_and_messages: sort_key_and_messages[0])
    print([sort_key for sort_key, _ in sort_keys_and_list_of_messages])
    
    list_of_messages = [messages for _, messages in sort_keys_and_list_of_messages]
    return list_of_messages

In [5]:
def get_correct_answer(question):
    if 'Three airline' in question: return 79
    if 'Fred and George' in question: return 250
    if 'Triangle $ABC$' in question: return 180
    if 'Find the three' in question: return 143
    if 'We call a' in question: return 3
    if 'Let $ABC$ be' in question: return 751
    if 'For a positive' in question: return 891
    if 'For positive integers' in question: return 810
    if 'The Fibonacci numbers' in question: return 201
    if 'Alice writes all' in question: return 902
    return 0

In [6]:
def create_starter_messages(question, index):
    options = []
    # for _ in range(13):
    #     options.append(
    #         [
    #             {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step. Return final answer within \\boxed{}, after taking modulo 1000."},
    #             {"role": "user", "content": question},
    #         ]
    #     )
    # for _ in range(3):    
    #     options.append(
    #         [
    #             {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step. After you get your final answer, take modulo 1000, and return the final answer within \\boxed{}."},
    #             {"role": "user", "content": question},
    #         ],
    #     )
    # for _ in range(2):
    #     options.append(
    #         [
    #             {"role": "user", "content": "Please solve the problems with deep resoning. You are careful and always recheck your conduction. You will never give answer directly until you have enough confidence. You should think step-by-step. \
    #                                         Return final answer within \\boxed{}, after taking modulo 1000. \n" + question},
    #         ]
    #     )
    # for _ in range(2):
    #     options.append(
    #         [
    #             {"role": "user", "content": "Please approach problems through a comprehensive cycle of analysis, summarization, exploration, reassessment, reflection, backtracking, and iteration, fostering a thoughtful and well-considered problem-solving process. \
    #                                         Put your final answer in \\boxed{}. If the answer exceeds 1000, take the answer modulo 1000.\n" + question},
    #         ]
    #     )
    # for _ in range(2):
    #     options.append(
    #         [
    #             {"role": "user", "content": "Break the problem into sub-steps, reasoning at each stage, and give the final answer in the format \\boxed{}. If the answer exceeds 1000, take the answer modulo 1000. \n" + question},
    #         ]
    #     )
    # for _ in range(2):
    #     options.append(
    #         [
    #             {"role": "user", "content": "You should think step-by-step and you are good at reverse thinking to recheck your answer and fix all possible mistakes. After you get your final answer, take modulo 1000, and return the final answer within \\boxed{}. " + question},
    #         ]
    #     )
    # for _ in range(2):
    #     options.append(
    #         [
    #             {"role": "user", "content": "Please carefully read the problem statement first to ensure you fully understand its meaning and key points. Then, solve the problem correctly and completely through deep reasoning. Finally, return the result modulo 1000 and enclose it in \\boxed{}." + question},
    #         ]
    #     )
    # for _ in range(2):
    #     options.append(
    #         [
    #             {"role": "user", "content": "Break the problem into sub-steps, reasoning at each stage, and give the final answer in the format \\boxed{}. If the answer exceeds 1000, take the answer modulo 1000. \n" + question},
    #         ]
    #     )
    # for _ in range(2):
    #     options.append(
    #         [
    #             {"role": "user", "content": "Please approach problems through a comprehensive cycle of analysis, summarization, exploration, reassessment, reflection, backtracking, and iteration, fostering a thoughtful and well-considered problem-solving process. \
    #                                         Put your final answer in \\boxed{}. If the answer exceeds 1000, take the answer modulo 1000.\n" + question},
    #         ]
    #     )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """Process the problem through isomorphic transformation:
a) Establish equivalent representations in different mathematical domains
b) Solve in the most advantageous domain using vertical depth-first search
c) Map solution back to original problem space
d) Validate isomorphism preservation through bidirectional mapping
e) Execute convergence testing on numerical results
If the final answer exceeds 1000, take the final answer modulo 1000.
Final answer (0-999) in \\boxed{} after cross-domain confirmation. \n
""" + question},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """Employ operational stack methodology:
1. Layer 1: Pattern recognition and theorem mapping
2. Layer 2: Lemma decomposition and dependency resolution
3. Layer 3: Atomic operation execution with rollback capability
4. Integrity check: Proof-tree validation
5. Optimize: Perform minimal sufficient computation proof
Return \\boxed{answer} with value mod 1000 after stack-wide verification. \n
""" + question},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """Analyze this problem through first-principles reasoning.
1) Decompose into core components
2) Identify relevant theorems/strategies
3) Formulate key conjectures with verification checkpoints
4) Execute solution with cross-validation at critical steps
5) Perform boundary case analysis
6) Document solution path concisely
After you get your final answer, take modulo 1000, and return the final answer within \\boxed{}. Prioritize quality over explanation length. \n
""" + question},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """Solve this IMO-level problem using strategic phase transitions:
Phase 1: Problem comprehension and pattern mapping (1 mental cycle)
Phase 2: Dimensional reduction through invariant identification
Phase 3: Backward-forward search with heuristic pruning
Phase 4: Convergence verification through dual methods verification
Return only the final verified answer modulo 1000: \\boxed{}. \n
""" + question},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """Execute these steps strictly:
1. Cryptographic scan of problem statement
2. Multivariate analysis with combinatorial sanitation
3. Apply hybrid methods (analytic + algebraic topology where appropriate)
4. Error detection via inverse problem reconstruction
5. Precision result formatting
If the final answer exceeds 1000, take the final answer modulo 1000.
Final numeric answer (0-999): \\boxed{}. \n
""" + question},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """Address this problem through constraints decomposition:
- Isolate key variables and their relationships
- Develop solution pathways for distinct constraint subsets
- Synthesize partial solutions using combinatorial logic
- Perform dimensional analysis on numerical results
- Apply sanity checks through limit case evaluation
Return final answer modulo 1000 in \\boxed{} after comprehensive verification. \n
""" + question},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """"As an expert mathematical problem solver, systematically analyze the problem through these stages:
1. Problem Breakdown: Identify core components and hidden constraints
2. Strategy Formulation: Propose multiple approaches with theoretical justification
3. Tactical Execution: Implement chosen method with rigorous logical deductions
4. Critical Verification: Perform sanity checks and alternative path validation
5. Precision Refinement: Ensure technical accuracy at every computational step
Present modular reasoning at each stage. Finalize by reporting the final answer modulo 1000 within \\boxed{}. \n
""" + question},
            ]
        )
    for _ in range(2):
        options.append(
            [
                {"role": "user", "content": """Solve this mathematical problem using deep theoretical analysis:
- Frame the problem within relevant mathematical domains
- Identify applicable theorems/lemmas with explicit citations
- Develop formal proofs for non-trivial steps
- Examine edge cases and boundary conditions
- Quantitatively verify dimensional consistency
Execute reverse-engineering verification before finalizing. Present conclusion modulo 1000 in \\boxed{}. \n
""" + question},
            ]
        )
    return options[index%len(options)]

g_score = 0
g_count = 0
def predict_for_question(question: str) -> int:
    # selected_questions_only = True
    # #selected_questions_only = False
    # if selected_questions_only and not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    #     #if "Triangle" not in question:
    #     #    return 210
    #     if "Triangle" not in question and "delightful" not in question and "George" not in question:
    #         return 210

    global g_score
    global g_count
    if time.time() > cutoff_time:
        return 210
    
    print(question)

    num_seqs = MAX_NUM_SEQS
    if time.time() > cutoff_times[-1]:
        num_seqs = 2 * MAX_NUM_SEQS // 3
    
    list_of_messages = [create_starter_messages(question, index) for index in range(num_seqs)]

    all_extracted_answers = []
    for _ in range(1):
        list_of_messages = batch_message_generate(list_of_messages)
        
        if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
            df = pd.DataFrame(
                {
                    "question": [question] * len(list_of_messages),
                    "message": [messages[-1]["content"] for messages in list_of_messages],
                }
            )
            df.to_csv(f"{str(int(time.time() - start_time)).zfill(5)}.csv", index=False)
        
        list_of_messages, extracted_answers = batch_message_filter(list_of_messages)
        all_extracted_answers.extend(extracted_answers)
    
    print(all_extracted_answers)
    answer = select_answer(all_extracted_answers)
    correct_answer = get_correct_answer(question)
    print(answer)
    if str(answer) == str(correct_answer):
        g_score += 1
    g_count += 1
    print("\n\n")
    cutoff_times.pop()
    print(f"score: {g_score}/{g_count}")
    return answer

def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)
    question = question.item(0)
    answer = predict_for_question(question)
    print(question)
    print("------\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [7]:
#predict_for_question("Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?")

In [8]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [9]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
#            '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv',
            'reference.csv',
        )
    )

------
71beb6
For a positive integer $n$, let $S(n)$ denote the sum of the digits of $n$ in base 10. Compute $S(S(1)+S(2)+\cdots+S(N))$ with $N=10^{100}-2$.


Processed prompts: 100%|██████████| 32/32 [05:41<00:00, 10.66s/it, est. speed input: 14.27 toks/s, output: 868.86 toks/s]


[12122, 12122, 12144, 5900, 12126, 7104, 5049, 11042, 8529, 8511, 10047, 10733, 11188, 12119, 12145, 9985, 12122, 12122, 5359, 5875, 4778, 10847, 12149, 8333, 12135, 9417, 5680, 7929, 5275, 8724, 8615, 6101]
[12122, 12122, 12144, 5900, 12126, 7104, 5049, 11042, 8529, 8511, 10047, 10733, 11188, 12119, 12145, 9985, 12122, 12122, 5359, 5875, 4778, 10847, 12149, 8333, 12135, 9417, 5680, 7929, 5275, 8724, 8615, 6101]
[4778, 5049, 5275, 5359, 5680, 5875, 5900, 6101, 7104, 7929, 8333, 8511, 8529, 8615, 8724, 9417, 9985, 10047, 10733, 10847, 11042, 11188, 12119, 12122, 12122, 12122, 12122, 12126, 12135, 12144, 12145, 12149]
['19', '917', '891', '891', '892', '891', '892', '891', '891', '891', '891', '26', '18', '891', '900', '35', '18', '18', '908', '900', '873', '899']
891



score: 1/1
For a positive integer $n$, let $S(n)$ denote the sum of the digits of $n$ in base 10. Compute $S(S(1)+S(2)+\cdots+S(N))$ with $N=10^{100}-2$.
------


------
a1d40b
The Fibonacci numbers are defined as follow

Processed prompts: 100%|██████████| 32/32 [03:37<00:00,  6.80s/it, est. speed input: 32.23 toks/s, output: 800.23 toks/s]


[6508, 5320, 6530, 5634, 5408, 4300, 5673, 6260, 6069, 4154, 4003, 5202, 2660, 5009, 3673, 4094, 5178, 6008, 6164, 6333, 5347, 3442, 5627, 4916, 5033, 5377, 5894, 5399, 5222, 5281, 10008, 8347]
[6508, 5320, 6530, 5634, 5408, 4300, 5673, 6260, 6069, 4154, 4003, 5202, 2660, 5009, 3673, 4094, 5178, 6008, 6164, 6333, 5347, 3442, 5627, 4916, 5033, 5377, 5894, 5399, 5222, 5281, 10008, 8347]
[2660, 3442, 3673, 4003, 4094, 4154, 4300, 4916, 5009, 5033, 5178, 5202, 5222, 5281, 5320, 5347, 5377, 5399, 5408, 5627, 5634, 5673, 5894, 6008, 6069, 6164, 6260, 6333, 6508, 6530, 8347, 10008]
['201', '201', '201', '201', '201', '201', '201', '201', '201', '201', '0', '200', '201', '201', '200', '202', '201', '201', '201', '201', '201', '202', '201', '201', '201', '200', '201', '201', '201', '201', '0', '201']
201



score: 2/2
The Fibonacci numbers are defined as follows: $F_0 = 0$, $F_1 = 1$, and $F_{n+1} = F_n + F_{n-1}$ for $n \geq 1$. There are $N$ positive integers $n$ strictly less than $10^{101}$

...

[12035, 12035, 12057, 12057, 12039, 12039, 12062, 12062, 7360, 12048, 12068, 12068, 12032, 12032, 12058, 12058, 12035, 12035, 12057, 12057, 12039, 12039, 12062, 12062, 12048, 7085, 12068, 12068, 12032, 12032, 12058, 12058]
[12035, 12035, 12057, 12057, 12039, 12039, 12062, 12062, 7360, 12048, 12068, 12068, 12032, 12032, 12058, 12058, 12035, 12035, 12057, 12057, 12039, 12039, 12062, 12062, 12048, 7085, 12068, 12068, 12032, 12032, 12058, 12058]
[7085, 7360, 12032, 12032, 12032, 12032, 12035, 12035, 12035, 12035, 12039, 12039, 12039, 12039, 12048, 12048, 12057, 12057, 12057, 12057, 12058, 12058, 12058, 12058, 12062, 12062, 12062, 12062, 12068, 12068, 12068, 12068]
['0', '0']
0



score: 5/8
For positive integers $x_1,\ldots, x_n$ define $G(x_1, \ldots, x_n)$ to be the sum of their $\frac{n(n-1)}{2}$ pairwise greatest common divisors. We say that an integer $n \geq 2$ is \emph{artificial} if there exist $n$ different positive integers $a_1, ..., a_n$ such that 
\[a_1 + \cdots + a_n = G(a_1,

Processed prompts: 100%|██████████| 32/32 [04:35<00:00,  8.60s/it, est. speed input: 17.22 toks/s, output: 630.11 toks/s]


[4386, 7221, 4407, 4767, 7601, 5052, 5691, 4142, 3347, 5169, 3654, 6708, 3420, 4038, 7282, 5195, 4326, 8030, 3094, 3792, 4887, 5485, 5730, 4622, 3122, 4567, 4856, 7416, 5746, 7506, 6035, 12149]
[4386, 7221, 4407, 4767, 7601, 5052, 5691, 4142, 3347, 5169, 3654, 6708, 3420, 4038, 7282, 5195, 4326, 8030, 3094, 3792, 4887, 5485, 5730, 4622, 3122, 4567, 4856, 7416, 5746, 7506, 6035, 12149]
[3094, 3122, 3347, 3420, 3654, 3792, 4038, 4142, 4326, 4386, 4407, 4567, 4622, 4767, 4856, 4887, 5052, 5169, 5195, 5485, 5691, 5730, 5746, 6035, 6708, 7221, 7282, 7416, 7506, 7601, 8030, 12149]
['143', '143', '143', '143', '101', '143', '143', '101', '143', '143', '143', '143', '143', '143', '143', '143', '143', '143', '143', '143', '143', '101', '143', '143', '143', '143', '143', '143', '143', '143', '143']
143



score: 6/9
Find the three-digit number $n$ such that writing any other three-digit number $10^{2024}$ times in a row and $10^{2024}+2$ times in a row results in two numbers divisible by $n$.
--

Processed prompts: 100%|██████████| 32/32 [06:31<00:00, 12.24s/it, est. speed input: 19.13 toks/s, output: 985.01 toks/s]


[12040, 12040, 12062, 12062, 12044, 12044, 12067, 12067, 12053, 12053, 12073, 12073, 12037, 12037, 12063, 12063, 12040, 12040, 12062, 12062, 12044, 12044, 12067, 12067, 12053, 12053, 12073, 12073, 12037, 12037, 12063, 12063]
[12040, 12040, 12062, 12062, 12044, 12044, 12067, 12067, 12053, 12053, 12073, 12073, 12037, 12037, 12063, 12063, 12040, 12040, 12062, 12062, 12044, 12044, 12067, 12067, 12053, 12053, 12073, 12073, 12037, 12037, 12063, 12063]
[12037, 12037, 12037, 12037, 12040, 12040, 12040, 12040, 12044, 12044, 12044, 12044, 12053, 12053, 12053, 12053, 12062, 12062, 12062, 12062, 12063, 12063, 12063, 12063, 12067, 12067, 12067, 12067, 12073, 12073, 12073, 12073]
[]
210



score: 6/10
Let $ABC$ be a triangle with $BC=108$, $CA=126$, and $AB=39$. Point $X$ lies on segment $AC$ such that $BX$ bisects $\angle CBA$. Let $\omega$ be the circumcircle of triangle $ABX$. Let $Y$ be a point on $\omega$ different from $X$ such that $CX=CY$. Line $XY$ meets $BC$ at $E$. The length of the segme